In [1]:
import sys
sys.path.append('/home/td/Documents/sports_predictor/nba')
sys.path.append('/home/td/Documents/sports_predictor')

from scipy import stats
import time
# import requests
from bs4 import BeautifulSoup
import threading
import pandas as pd
import tqdm
import lightgbm as lgb
from sklearn.preprocessing import StandardScaler, PowerTransformer
import numpy as np
import random
from sklearn import model_selection, metrics 
import pickle
import lightgbm
from nba.common import (
    timeit,
)
   


base_url = 'https://www.basketball-reference.com/'
day_scores_base_url = 'https://www.basketball-reference.com/boxscores/?month={month}&day={day}&year={year}'
data_path = r'/media/td/Samsung_T5/sports/nba'
# data_path = r'C:\Users\TristanDelforge\Documents\sports_predictor\nba'
db_name = 'nba_db'
box_score_link_table_name = 'boxscore_links'
general_feature_data_table_name = 'general_features'

box_score_details_table_name = 'boxscore_details'
processed_team_data_table_name = 'processed_team_data'
player_detail_table_name = 'player_details'
processed_player_data_table_name = 'processed_player_data'
aggregated_player_data_table_name = 'aggregated_player_data'
combined_feature_file_data_table_name = 'combined_feature_file'
past_n_game_dataset_table_name = 'past_n_game_dataset'
target = 'win'

date_record_pickle_file_name = 'scraped_dates'
box_score_record_pickle_file_name = 'scraped_games'
max_tries = 5
file_lock = threading.Lock()

starting_rating = 1000
rating_k_factor = 100
rating_floor = 100
rating_ceiling = 10000
rating_d = 1000
k_min_sensitivity = 1

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('display.max_colwidth', 1000)


In [2]:

class Model():

    def __init__(self, model_type, model_params, max_iter = 10000, early_stopping_rounds = 100):
        self.model_type = model_type
        self.model_params = model_params
        self.transformers_dict = dict()
        self.lightgbm_early_stopping_rounds = early_stopping_rounds
        self.lightgbm_max_iter = max_iter

    @timeit
    def fit(self, x, y):
        print('entered fit, x shape: {}'.format(x.shape))
        self.transformers_dict = dict()
        self.columns = x.columns

        if self.model_type == 'lightgbm':
            x_train, x_val, y_train, y_val = model_selection.train_test_split(x, y, random_state=1)
            lgtrain = lightgbm.Dataset(x_train, y_train)
            lgvalid = lightgbm.Dataset(x_val, y_val)

            self.model = lightgbm.train(
                self.model_params,
                lgtrain,
                num_boost_round=self.lightgbm_max_iter,
                valid_sets=[lgtrain, lgvalid],
                valid_names=['train', 'valid'],
                early_stopping_rounds=self.lightgbm_early_stopping_rounds,
                verbose_eval=100
            )

    @timeit
    def predict(self, x):
        if self.model_type == 'lightgbm':
            return self.model.predict(x, num_iteration=self.model.best_iteration)
            

    def evaluate(self):
        if self.model_type == 'lightgbm':
            output = []

            for i, j in zip(self.columns, self.model.feature_importance('gain', iteration=self.model.best_iteration)):
                output.append({'column': i, 'feature_importance': j})
            return pd.DataFrame.from_dict(output).sort_values('feature_importance', ascending=False)
        

In [3]:
from scipy import stats

import copy

lgbm_params_classification = {
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'metric': 'binary_error',
    "learning_rate": 0.01,
    "max_depth": -1,
    'num_leaves': 31,
    'seed':1,
    'bagging_seed':1,
    'feature_fraction_seed':1,
    'data_random_seed':1,
    "feature_fraction": 0.8,
    "bagging_fraction": 0.8,
    'bagging_freq': 1,
}

lgbm_params_regression = {
    'boosting_type': 'gbdt',
    'objective': 'l2',
    'metric': 'l1',
    "learning_rate": 0.01,
    "max_depth": -1,
    'num_leaves': 31,
    'seed':1,
    'bagging_seed':1,
    'feature_fraction_seed':1,
    'data_random_seed':1,
    "feature_fraction": 0.8,
    "bagging_fraction": 0.8,
    'bagging_freq': 1,
}



def evaluate_features(df, feature_list, target, target_type):
    if target_type == 'classification':
        params = copy.deepcopy(lgbm_params_classification)
    elif target_type == 'regression':
        params = copy.deepcopy(lgbm_params_regression)
    else:
        raise Exception('Invalid target type: {}'.format(target_type))
    
    results = dict()
    for i in feature_list:
        slope, intercept, r_value, p_value, std_err = stats.linregress(df[i], df[target])
        results[i] = {'slope':slope,
                     'intercept':intercept,
                     'r_value':r_value,
                     'r2_value':r_value*r_value,
                     'p_value':p_value,
                     'std_err':std_err}
    
    df_train, df_val = model_selection.train_test_split(df, random_state = 1)
    model = Model('lightgbm', params)
    model.fit(df_train[feature_list], df_train[target])
    
    preds = model.predict(df_val[feature_list])
    preds = np.rint(preds).astype(int)   
    
    if target_type == 'classification':
        model_score = metrics.accuracy_score(df_val[target], preds)
    if target_type == 'regression':
        model_score = metrics.r2_score(df_val[target], preds)
    
    feature_importance = model.evaluate()
    print(target_type, model_score)
    
    feature_importance['slope'] = feature_importance.apply(lambda x: results[x['column']]['slope'], axis = 1)
    feature_importance['intercept'] = feature_importance.apply(lambda x: results[x['column']]['intercept'], axis = 1)
    feature_importance['r_value'] = feature_importance.apply(lambda x: results[x['column']]['r_value'], axis = 1)
    feature_importance['r2_value'] = feature_importance.apply(lambda x: results[x['column']]['r2_value'], axis = 1)
    feature_importance['p_value'] = feature_importance.apply(lambda x: results[x['column']]['p_value'], axis = 1)
    feature_importance['std_err'] = feature_importance.apply(lambda x: results[x['column']]['std_err'], axis = 1)
    feature_importance['model_performance'] = model_score
    
    return feature_importance, preds


## Raw data

In [ ]:
team_df = pd.read_csv(f'{data_path}/{box_score_details_table_name}.csv', sep = '|', low_memory=False)
player_df = pd.read_csv(f'{data_path}/{player_detail_table_name}.csv', sep = '|', low_memory=False)


In [ ]:
team_df.shape, player_df.shape

In [ ]:
team_df.head()

In [ ]:
player_df.head()

## Player stats aggregated per game

In [ ]:
df_test = pd.DataFrame(data = [[1, 2, 3],
                              [1, 2, 3],
                              [1, 2, 3],
                              [2, 2, 3]],
                      columns = ['a', 'b', 'c'])


In [ ]:
df_test.shift(periods=1).rolling(window=2).mean()

##  Encoding evaluation

In [ ]:
feature_df_encoding = pd.read_csv(f'{data_path}/{general_feature_data_table_name}.csv', sep = '|')
feature_df_encoding = feature_df_encoding.set_index('key')



In [ ]:
feature_df_encoding.head()

In [ ]:
encodings_dict = dict()
encodings_dict[('pca', 8)] = pd.read_csv(f'{data_path}/encoded_file_loc_pca_8.csv', sep = '|')
encodings_dict[('pca', 32)] = pd.read_csv(f'{data_path}/encoded_file_loc_pca_32.csv', sep = '|')
encodings_dict[('pca', 128)] = pd.read_csv(f'{data_path}/encoded_file_loc_pca_128.csv', sep = '|')
encodings_dict[('pca', 512)] = pd.read_csv(f'{data_path}/encoded_file_loc_pca_512.csv', sep = '|')
encodings_dict[('dense_autoencoder', 8)] = pd.read_csv(f'{data_path}/encoded_file_loc_dense_autoencoder_8.csv', sep = '|')
encodings_dict[('dense_autoencoder', 32)] = pd.read_csv(f'{data_path}/encoded_file_loc_dense_autoencoder_32.csv', sep = '|')
encodings_dict[('dense_autoencoder', 128)] = pd.read_csv(f'{data_path}/encoded_file_loc_dense_autoencoder_128.csv', sep = '|')
encodings_dict[('dense_autoencoder', 512)] = pd.read_csv(f'{data_path}/encoded_file_loc_dense_autoencoder_512.csv', sep = '|')


In [ ]:

cols = [i for i in feature_df_encoding.columns if i not in ['win', 'score_diff', 'key']]
cols = list(set(cols))
baseline_classification, baseline_preds_classification = evaluate_features(feature_df_encoding, cols, 'win', 'classification')
baseline_regression, baseline_preds_regression = evaluate_features(feature_df_encoding, cols, 'score_diff', 'regression')

In [ ]:

for i in encodings_dict.keys():
    encodings_dict[i] = encodings_dict[i].set_index('key')
    

In [ ]:
feature_classification_df_dict_encoding = dict()
feature_classification_dfs_encoding = list()
classification_pred_df_encoding = pd.DataFrame()

feature_regression_df_dict_encoding = dict()
feature_regression_dfs_encoding = list()
regression_pred_df_encoding = pd.DataFrame()


for i in encodings_dict.keys():
    print(f'n: {i}')
    temp_df = encodings_dict[i].join(feature_df_encoding[['win', 'score_diff']])
    
    cols = [c for c in temp_df.columns if c not in ['win', 'score_diff']]
    temp_df_classification, preds_classification = evaluate_features(temp_df, cols, 'win', 'classification')
    temp_df_regression, preds_regression = evaluate_features(temp_df, cols, 'score_diff', 'regression')

    print(temp_df.shape, preds_classification.shape, preds_regression.shape)
    
    temp_df_classification['dim_reduction_algorithm'] = i[0]
    temp_df_classification['latent_dims'] = i[1]
    classification_pred_df_encoding[i] = preds_classification
    feature_classification_df_dict_encoding[i] = temp_df_classification
    feature_classification_dfs_encoding.append(temp_df_classification)
    
    temp_df_regression['dim_reduction_algorithm'] = i[0]
    temp_df_regression['latent_dims'] = i[1]
    regression_pred_df_encoding[n] = preds_regression
    feature_regression_df_dict_encoding[i] = temp_df_regression
    feature_regression_dfs_encoding.append(temp_df_regression)
    

In [ ]:
pd.concat(feature_regression_dfs_encoding).groupby(['latent_dims', 'dim_reduction_algorithm'])['model_performance'].mean()

In [ ]:
pd.concat(feature_classification_dfs_encoding).groupby(['latent_dims', 'dim_reduction_algorithm'])['model_performance'].mean()

In [ ]:
pd.concat(feature_regression_dfs_encoding).sort_values('r2_value', ascending = False).head(20)

## Feature evaluation

In [4]:
feature_df = pd.read_csv(f'{data_path}/{general_feature_data_table_name}2.csv', sep = '|', low_memory = False)
feature_df.shape

(17744, 29293)

In [5]:
feature_df.head()

,team_pregame_rating_0,days_since_last_match,home,year,month,team_aggregate_past_50_game_skew_player_stats_aggregated_by_game_trb_pct_median,team_aggregate_past_100_game_avg_player_stats_aggregated_by_game_stl_skew,team_aggregate_past_5_game_median_player_stats_aggregated_by_game_stl_pct_avg,team_aggregate_past_20_game_max_player_stats_aggregated_by_game_fg_min,team_aggregate_past_50_game_avg_player_stats_aggregated_by_game_fg_pct_var,team_aggregate_past_40_game_avg_player_stats_aggregated_by_game_ts_pct_max,team_aggregate_past_7_game_avg_player_stats_aggregated_by_game_ast_pct_var,team_aggregate_past_12_game_max_player_stats_aggregated_by_game_ts_pct_skew,team_aggregate_past_5_game_max_player_stats_aggregated_by_game_ts_pct_max,team_aggregate_past_20_game_avg_ft,team_aggregate_past_100_game_var_player_stats_aggregated_by_game_mp_avg,team_aggregate_past_1_game_avg_player_stats_aggregated_by_game_pts_var,team_aggregate_past_40_game_min_player_stats_aggregated_by_game_ast_pct_avg,team_aggregate_past_75_game_var_player_stats_aggregated_by_game_tov_pct_avg,team_aggregate_past_3_game_max_player_stats_aggregated_by_game_fg3a_per_fga_pct_min,team_aggregate_past_30_game_max_player_stats_aggregated_by_game_fg3a_per_fga_pct_min,team_aggregate_past_30_game_max_player_stats_aggregated_by_game_fga_avg,team_aggregate_past_20_game_skew_player_stats_aggregated_by_game_fg3_avg,team_aggregate_past_30_game_median_player_stats_aggregated_by_game_fta_median,team_aggregate_past_7_game_avg_player_stats_aggregated_by_game_fg_median,team_aggregate_past_75_game_var_fga,team_aggregate_past_12_game_avg_player_stats_aggregated_by_game_ft_median,team_aggregate_past_100_game_max_fta,team_aggregate_past_50_game_var_efg_pct,team_aggregate_past_7_game_avg_player_stats_aggregated_by_game_def_rtg_max,team_aggregate_past_75_game_median_player_stats_aggregated_by_game_drb_pct_avg,team_aggregate_past_5_game_max_player_stats_aggregated_by_game_orb_avg,team_aggregate_past_7_game_var_player_stats_aggregated_by_game_plus_minus_avg,team_aggregate_past_75_game_skew_player_stats_aggregated_by_game_pts_avg,team_aggregate_past_50_game_min_player_stats_aggregated_by_game_stl_pct_avg,team_aggregate_past_40_game_var_player_stats_aggregated_by_game_fg3_pct_max,team_aggregate_past_30_game_min_player_stats_aggregated_by_game_orb_pct_var,team_aggregate_past_12_game_skew_player_stats_aggregated_by_game_ft_pct_max,team_aggregate_past_12_game_min_fg_pct,team_aggregate_past_12_game_var_drb_pct,team_aggregate_past_50_game_max_player_stats_aggregated_by_game_fga_skew,team_aggregate_past_40_game_max_player_stats_aggregated_by_game_plus_minus_max,team_aggregate_past_5_game_median_player_stats_aggregated_by_game_trb_avg,team_aggregate_past_3_game_skew_player_stats_aggregated_by_game_ast_max,team_aggregate_past_20_game_min_player_stats_aggregated_by_game_pf_min,team_aggregate_past_100_game_skew_player_stats_aggregated_by_game_usg_pct_avg,team_aggregate_past_50_game_avg_player_stats_aggregated_by_game_plus_minus_skew,team_aggregate_past_20_game_avg_player_stats_aggregated_by_game_ft_pct_min,team_aggregate_past_12_game_avg_fg3a_per_fga_pct,team_aggregate_past_40_game_min_fg3_pct,team_aggregate_past_20_game_min_player_stats_aggregated_by_game_orb_var,team_aggregate_past_100_game_skew_tov_pct,team_aggregate_past_40_game_skew_player_stats_aggregated_by_game_trb_max,team_aggregate_past_100_game_min_player_stats_aggregated_by_game_fg3a_per_fga_pct_skew,team_aggregate_past_5_game_avg_player_stats_aggregated_by_game_orb_max,team_aggregate_past_40_game_median_ft_pct,team_aggregate_past_75_game_skew_player_stats_aggregated_by_game_tov_median,team_aggregate_past_75_game_avg_player_stats_aggregated_by_game_trb_max,team_aggregate_past_5_game_var_days_since_last_match,team_aggregate_past_50_game_skew_fg3_pct,team_aggregate_past_3_game_median_fg3a_per_fga_pct,team_aggregate_past_40_game_avg_player_stats_aggregated_by_game_mp_skew,team_aggregate_past_100_game_skew_score_diff,team_aggregate_past_12_game_max_

In [6]:

cols = [i for i in feature_df.columns if i not in ['win', 'score_diff', 'key']]
cols = list(set(cols))
baseline_classification, baseline_preds_classification = evaluate_features(feature_df, cols, 'win', 'classification')
baseline_regression, baseline_preds_regression = evaluate_features(feature_df, cols, 'score_diff', 'regression')


/home/td/anaconda3/envs/sports_predictor/lib/python3.7/site-packages/scipy/stats/_stats_mstats_common.py:130: RuntimeWarning: invalid value encountered in double_scalars
  slope = r_num / ssxm
/home/td/anaconda3/envs/sports_predictor/lib/python3.7/site-packages/scipy/stats/_stats_mstats_common.py:142: RuntimeWarning: divide by zero encountered in double_scalars
  sterrest = np.sqrt((1 - r**2) * ssym / ssxm / df)


function: 'fit' starting
entered fit, x shape: (13308, 29290)
Training until validation scores don't improve for 100 rounds
[100]	train's binary_error: 0.220319	valid's binary_error: 0.333934
Early stopping, best iteration is:
[89]	train's binary_error: 0.222322	valid's binary_error: 0.329426
function: 'fit' finished in  176.26 seconds
function: 'predict' starting
function: 'predict' finished in  15.81 seconds
classification 0.657799819657349
function: 'fit' starting
entered fit, x shape: (13308, 29290)
Training until validation scores don't improve for 100 rounds
[100]	train's l1: 9.54662	valid's l1: 10.2109
[200]	train's l1: 8.78276	valid's l1: 9.96933
[300]	train's l1: 8.21936	valid's l1: 9.93184
[400]	train's l1: 7.72592	valid's l1: 9.91954
[500]	train's l1: 7.27225	valid's l1: 9.92157
Early stopping, best iteration is:
[447]	train's l1: 7.50659	valid's l1: 9.91621
function: 'fit' finished in  535.93 seconds
function: 'predict' starting
function: 'predict' finished in  16.17 second

In [7]:
baseline_classification.sort_values('feature_importance', ascending = False).head(10)

,column,feature_importance,slope,intercept,r_value,r2_value,p_value,std_err,model_performance
16626,team_pregame_rating_0_diff_vs_opponent_feature,23338.474619,0.000174,0.500000,0.318186,0.101242,0.000000e+00,0.000004,0.6578
20795,home_diff_vs_opponent_feature,6127.911209,0.087889,0.500000,0.173594,0.030135,4.500050e-120,0.003743,0.6578
12739,home,4920.312504,0.171498,0.415846,0.171468,0.029401,3.731829e-117,0.007398,0.6578
20169,team_aggregate_past_75_game_skew_win_diff_vs_opponent_feature,1863.098206,-0.181192,0.500000,-0.322366,0.103920,0.000000e+00,0.003994,0.6578
19101,team_aggregate_past_75_game_avg_score_diff_diff_vs_opponent_feature,1795.065804,0.028480,0.500000,0.332160,0.110330,0.000000e+00,0.000607,0.6578
11109,team_aggregate_past_50_game_avg_player_stats_aggregated_by_game_plus_minus_avg_diff_vs_opponent_feature,1586.176395,0.067751,0.500000,0.332400,0.110490,0.000000e+00,0.001443,0.6578
12212,team_aggregate_past_75_game_avg_player_stats_aggregated_by_game_plus_minus_avg_diff_vs_opponent_feature,1358.763294,0.072195,0.500000,0.332520,0.110569,0.000000e+00,0.001537,0.6578
1681,team_aggregate_past_20_game_avg_player_stats_aggregated_by_game_plus_minus_avg_diff_vs_opponent_feature,1081.261805,0.054935,0.500000,0.320475,0.102704,0.000000e+00,0.001219,0.6578
22796,team_aggregate_past_50_game_avg_player_stats_aggregated_by_game_plus_minus_max_diff_vs_opponent_feature,877.824085,0.038564,0.500000,0.295717,0.087448,0.000000e+00,0.000935,0.6578
1564,team_aggregate_past_75_game_avg_win_diff_vs_opponent_feature,798.901001,0.733262,0.500000,0.305657,0.093426,0.000000e+00,0.017148,0.6578


In [8]:
baseline_regression.sort_values('feature_importance', ascending = False).head(10)

,column,feature_importance,slope,intercept,r_value,r2_value,p_value,std_err,model_performance
16626,team_pregame_rating_0_diff_vs_opponent_feature,5.484332e+06,0.005317,-1.744148e-17,0.348277,0.121297,0.000000e+00,0.000107,0.185651
20795,home_diff_vs_opponent_feature,1.775406e+06,2.763435,0.000000e+00,0.195350,0.038162,3.846113e-152,0.104156,0.185651
12739,home,1.382218e+06,5.392307,-2.646011e+00,0.192957,0.037232,2.042320e-148,0.205861,0.185651
19101,team_aggregate_past_75_game_avg_score_diff_diff_vs_opponent_feature,5.448715e+05,0.873653,2.239018e-17,0.364672,0.132985,0.000000e+00,0.016747,0.185651
23032,team_aggregate_past_75_game_avg_player_stats_aggregated_by_game_plus_minus_median_diff_vs_opponent_feature,4.700849e+05,2.836338,-1.817258e-17,0.359837,0.129483,0.000000e+00,0.055213,0.185651
1343,team_aggregate_past_50_game_avg_player_stats_aggregated_by_game_plus_minus_median_diff_vs_opponent_feature,4.258945e+05,2.635551,8.443063e-18,0.359781,0.129442,0.000000e+00,0.051313,0.185651
10220,team_aggregate_past_30_game_median_player_stats_aggregated_by_game_plus_minus_avg_diff_vs_opponent_feature,3.938109e+05,1.643091,2.105475e-17,0.355320,0.126252,0.000000e+00,0.032451,0.185651
9802,team_aggregate_past_12_game_avg_score_diff_diff_vs_opponent_feature,3.793306e+05,0.582420,-1.492640e-17,0.345381,0.119288,0.000000e+00,0.011881,0.185651
20020,team_aggregate_past_50_game_skew_win_diff_vs_opponent_feature,3.398416e+05,-5.010239,3.210089e-17,-0.352085,0.123964,0.000000e+00,0.099993,0.185651
11109,team_aggregate_past_50_game_avg_player_stats_aggregated_by_game_plus_minus_avg_diff_vs_opponent_feature,3.304014e+05,2.102661,-1.347187e-17,0.369214,0.136319,0.000000e+00,0.039734,0.185651


In [9]:
import re

def extract_window_size(s):
    if 'team_aggregate_past_' in s:
        return int(re.findall('[0-9]+', s)[0])
    
    
def extract_team_aggregation_function(s):
    if '_player_stats_aggregated_by_game_' in s:
        s = s.split('_player_stats_aggregated_by_game_')[0]
    for i in ['avg', 'var', 'skew', 'max', 'median', 'min']:
        loc = s.find(i)
        if loc > -1:
            return i
        
    
def extract_player_aggregation_function(s):
    if '_player_stats_aggregated_by_game_' in s:
        s = s.split('_player_stats_aggregated_by_game_')[1]
        for i in ['avg', 'var', 'skew', 'max', 'median', 'min']:
            loc = s.find(i)
            if loc > -1:
                return i
    
    
def extract_players_stat_type(s):
    if '_player_stats_aggregated_by_game_' in s:
        s = s.split('_player_stats_aggregated_by_game_')[1]
        for i in ['_avg', '_var', '_skew', '_max', '_median', '_min']:
            if i in s:
                s = s.split(i)[0]
                return s
            
def extract_stat_type(s):
    if '_aggregated_by_game_' in s:
        s = s.split('_aggregated_by_game_')[1]
        for i in ['_avg', '_var', '_skew', '_max', '_median', '_min']:
            if i in s:
                s = s.split(i)[0]
                return s


def get_if_diff_feature(s):
    if 'diff_vs_opponent' in s:
        return 1
    return 0
            

def get_feature_type(s):
    if 'player_stats_aggregated_by_game' in s:
        return 'player_aggregate'
    if 'team_aggregate_' in s:
        return 'team_aggregate'
    if 'team_pregame_rating' in s or 'home' in s or 'days_since_last_match' in s or 'month' in s or 'year' in s:
        return 'general'
    

    
total_feature_importance = baseline_classification['feature_importance'].sum()
baseline_classification['normalized_feature_importance'] = baseline_classification['feature_importance'] / total_feature_importance

baseline_classification['aggregation_window_size'] = baseline_classification['column'].apply(extract_window_size)
baseline_classification['team_aggregation_function'] = baseline_classification['column'].apply(extract_team_aggregation_function)
baseline_classification['player_aggregation_function'] = baseline_classification['column'].apply(extract_player_aggregation_function)
baseline_classification['stat_type'] = baseline_classification['column'].apply(extract_stat_type)
baseline_classification['diff_feature'] = baseline_classification['column'].apply(get_if_diff_feature)
baseline_classification['feature_type'] = baseline_classification['column'].apply(get_feature_type)


In [ ]:
baseline_classification[(baseline_classification['column'] == 'team_aggregate_past_75_game_median_player_stats_aggregated_by_game_plus_minus_avg_diff_vs_opponent_feature')]

In [ ]:
baseline_classification.isna().sum()

In [ ]:
baseline_classification[baseline_classification['team_aggregation_function'].isna()]

In [10]:
player_aggregate_stat_choices = ['ast', 'ast_pct', 'blk', 'blk_pct', 'def_rtg', 'drb', 'drb_pct',
                                   'efg_pct', 'fg', 'fg3', 'fg3_pct', 'fg3a', 'fg3a_per_fga_pct',
                                   'fg_pct', 'fga', 'ft', 'ft_pct', 'fta', 'fta_per_fga_pct',
                                   'mp', 'off_rtg', 'orb', 'orb_pct', 'pf', 'plus_minus', 'pts',
                                   'stl', 'stl_pct', 'tov', 'tov_pct', 'trb', 'trb_pct', 'ts_pct',
                                   'usg_pct']

team_aggregate_stat_choices = ['ast', 'ast_pct', 'blk', 'blk_pct', 'def_rtg', 'drb', 'drb_pct', 'efg_pct',
                                      'fg', 'fg3', 'fg3_pct', 'fg3a', 'fg3a_per_fga_pct', 'fg_pct', 'fga', 'ft',
                                      'ft_pct', 'fta', 'fta_per_fga_pct', 'off_rtg', 'orb', 'orb_pct', 'pf',
                                      'pts', 'stl', 'stl_pct', 'tov', 'tov_pct', 'trb', 'trb_pct',
                                      'ts_pct', 'usg_pct', 'home', 'win', 'days_since_last_match', 'score_diff']

aggregation_window_sizes = [1, 3, 5, 7, 12, 20, 30, 40, 50, 75, 100]
team_aggregation_functions = ['avg', 'var', 'skew', 'max', 'median', 'min']
player_aggregation_functions = ['avg', 'var', 'skew', 'max', 'median', 'min']



def get_random_params_given_col_list(feature_importance_df, col_obj):

    
    feature_importance_df_team = feature_importance_df[feature_importance_df['feature_type'] == 'team_aggregate']
    feature_importance_df_player = feature_importance_df[feature_importance_df['feature_type'] == 'player_aggregate']
    feature_importance_df_general = feature_importance_df[feature_importance_df['feature_type'] == 'general']
    
    output_columns = feature_importance_df_general['column'].tolist()
    
    team_cols = feature_importance_df_team[(feature_importance_df_team['stat_type'].isin(col_obj['team_aggregate_stat']))&
                          (feature_importance_df_team['aggregation_window_size'].isin(col_obj['aggregation_windows']))&
                          (feature_importance_df_team['team_aggregation_function'].isin(col_obj['team_aggregation_functions']))]['column'].tolist()
    
    player_cols = feature_importance_df_player[(feature_importance_df_player['stat_type'].isin(col_obj['player_aggregate_stat']))&
                          (feature_importance_df_player['aggregation_window_size'].isin(col_obj['aggregation_windows']))&
                          (feature_importance_df_player['team_aggregation_function'].isin(col_obj['team_aggregation_functions']))&\
                           (feature_importance_df_player['player_aggregation_function'].isin(col_obj['player_aggregation_functions']))]['column'].tolist()
    
    if col_obj['use_team_data']:
        output_columns.extend(team_cols)
    if col_obj['use_player_data']:
        output_columns.extend(player_cols)
    return {'player_aggregate_stat':col_obj['player_aggregate_stat'],
           'team_aggregate_stat':col_obj['team_aggregate_stat'],
           'aggregation_windows':col_obj['aggregation_windows'],
           'team_aggregation_functions':col_obj['team_aggregation_functions'],
           'player_aggregation_functions':col_obj['player_aggregation_functions'],
           'output_columns':output_columns,
           'use_team_data':col_obj['use_team_data'],
           'use_player_data':col_obj['use_player_data'],
           'manual_entry':1}


def get_random_params_and_col_list(feature_importance_df):
#     use_team_data = random.choice([1, 0])
#     use_player_data = random.choice([1, 0])
    use_team_data = 1
    use_player_data = 1
    
    player_aggregate_stat_num = random.randint(1, len(player_aggregate_stat_choices))
    team_aggregate_stat_num = random.randint(1, len(team_aggregate_stat_choices))
    aggregation_window_size_num = random.randint(1, len(aggregation_window_sizes))
    team_aggregation_function_num = random.randint(1, len(team_aggregation_functions))
    player_aggregation_function_num = random.randint(1, len(player_aggregation_functions))
    
    player_aggregate_stat_final_choice = random.sample(player_aggregate_stat_choices, player_aggregate_stat_num)
    team_aggregate_stat_final_choice = random.sample(team_aggregate_stat_choices, team_aggregate_stat_num)
    aggregation_window_final_choice = random.sample(aggregation_window_sizes, aggregation_window_size_num)
    team_aggregation_function_final_choice = random.sample(team_aggregation_functions, team_aggregation_function_num)
    player_aggregation_function_final_choice = random.sample(player_aggregation_functions, player_aggregation_function_num)
    
#     player_aggregate_stat_final_choice = player_aggregate_stat_choices
#     team_aggregate_stat_final_choice = team_aggregate_stat_choices
#     aggregation_window_final_choice = aggregation_window_sizes
#     team_aggregation_function_final_choice = team_aggregation_functions
#     player_aggregation_function_final_choice = player_aggregation_functions
    
    
    feature_importance_df_team = feature_importance_df[feature_importance_df['feature_type'] == 'team_aggregate']
    feature_importance_df_player = feature_importance_df[feature_importance_df['feature_type'] == 'player_aggregate']
    feature_importance_df_general = feature_importance_df[feature_importance_df['feature_type'] == 'general']
    
    output_columns = feature_importance_df_general['column'].tolist()
    
    team_cols = feature_importance_df_team[(feature_importance_df_team['stat_type'].isin(team_aggregate_stat_final_choice))&
                          (feature_importance_df_team['aggregation_window_size'].isin(aggregation_window_final_choice))&
                          (feature_importance_df_team['team_aggregation_function'].isin(team_aggregation_function_final_choice))]['column'].tolist()
    
    player_cols = feature_importance_df_player[(feature_importance_df_player['stat_type'].isin(player_aggregate_stat_final_choice))&
                          (feature_importance_df_player['aggregation_window_size'].isin(aggregation_window_final_choice))&
                          (feature_importance_df_player['team_aggregation_function'].isin(team_aggregation_function_final_choice))&\
                           (feature_importance_df_player['player_aggregation_function'].isin(player_aggregation_function_final_choice))]['column'].tolist()
    
    if use_team_data:
        output_columns.extend(team_cols)
    if use_player_data:
        output_columns.extend(player_cols)
    return {'player_aggregate_stat':player_aggregate_stat_final_choice,
           'team_aggregate_stat':team_aggregate_stat_final_choice,
           'aggregation_windows':aggregation_window_final_choice,
           'team_aggregation_functions':team_aggregation_function_final_choice,
           'player_aggregation_functions':player_aggregation_function_final_choice,
           'output_columns':output_columns,
           'use_team_data':use_team_data,
           'use_player_data':use_player_data}

    

In [11]:
results = list()

In [ ]:


baseline_classification.to_csv('/media/td/Samsung_T5/sports/nba/baseline_classification2.csv', index = False, sep = '|')
    

In [19]:
import json

results = []
with open('/tmp/next_params.json', 'r') as f:
    possible_solutions = json.load(f)

for n, p in enumerate(possible_solutions):
    print(f'Iteration: {n}, Total number of results: {len(results)}')
    col_choice_dict = get_random_params_given_col_list(baseline_classification, p)
    temp_classification, _ = evaluate_features(feature_df, col_choice_dict['output_columns'], 'win', 'classification')
    temp_regression, _ = evaluate_features(feature_df, col_choice_dict['output_columns'], 'score_diff', 'regression')
    
    col_choice_dict['win_loss_accuracy'] = temp_classification['model_performance'].min()
    col_choice_dict['points_estimation_r2'] = temp_regression['model_performance'].min()
    col_choice_dict['col_count'] = len(col_choice_dict['output_columns'])
    
    del col_choice_dict['output_columns']

    results.append(col_choice_dict)
    pd.DataFrame.from_dict(results).sort_values('win_loss_accuracy', ascending = False).to_csv('/media/td/Samsung_T5/sports/nba/feature_reduction5.csv', index = False, sep = '|')
    
    

Iteration: 0, Total number of results: 0


/home/td/anaconda3/envs/sports_predictor/lib/python3.7/site-packages/scipy/stats/_stats_mstats_common.py:130: RuntimeWarning: invalid value encountered in double_scalars
  slope = r_num / ssxm
/home/td/anaconda3/envs/sports_predictor/lib/python3.7/site-packages/scipy/stats/_stats_mstats_common.py:142: RuntimeWarning: divide by zero encountered in double_scalars
  sterrest = np.sqrt((1 - r**2) * ssym / ssxm / df)


function: 'fit' starting
entered fit, x shape: (13308, 1290)
Training until validation scores don't improve for 100 rounds
[100]	train's binary_error: 0.245567	valid's binary_error: 0.335137
Early stopping, best iteration is:
[35]	train's binary_error: 0.25779	valid's binary_error: 0.329426
function: 'fit' finished in  6.95 seconds
function: 'predict' starting
function: 'predict' finished in  0.22 seconds
classification 0.6679440937781785
function: 'fit' starting
entered fit, x shape: (13308, 1290)
Training until validation scores don't improve for 100 rounds
[100]	train's l1: 9.67283	valid's l1: 10.2028
[200]	train's l1: 9.0441	valid's l1: 9.95265
[300]	train's l1: 8.61469	valid's l1: 9.89496
[400]	train's l1: 8.24515	valid's l1: 9.88999
[500]	train's l1: 7.89912	valid's l1: 9.89148
Early stopping, best iteration is:
[445]	train's l1: 8.08444	valid's l1: 9.88281
function: 'fit' finished in  34.97 seconds
function: 'predict' starting
function: 'predict' finished in  0.23 seconds
regres

/home/td/anaconda3/envs/sports_predictor/lib/python3.7/site-packages/scipy/stats/_stats_mstats_common.py:130: RuntimeWarning: invalid value encountered in double_scalars
  slope = r_num / ssxm
/home/td/anaconda3/envs/sports_predictor/lib/python3.7/site-packages/scipy/stats/_stats_mstats_common.py:142: RuntimeWarning: divide by zero encountered in double_scalars
  sterrest = np.sqrt((1 - r**2) * ssym / ssxm / df)


function: 'fit' starting
entered fit, x shape: (13308, 406)
Training until validation scores don't improve for 100 rounds
[100]	train's binary_error: 0.256487	valid's binary_error: 0.330628
Early stopping, best iteration is:
[90]	train's binary_error: 0.257289	valid's binary_error: 0.327021
function: 'fit' finished in  4.35 seconds
function: 'predict' starting
function: 'predict' finished in  0.08 seconds
classification 0.6638863841298467
function: 'fit' starting
entered fit, x shape: (13308, 406)
Training until validation scores don't improve for 100 rounds
[100]	train's l1: 9.7346	valid's l1: 10.2126
[200]	train's l1: 9.16952	valid's l1: 9.97921
[300]	train's l1: 8.78957	valid's l1: 9.91835
[400]	train's l1: 8.46203	valid's l1: 9.90121
[500]	train's l1: 8.16419	valid's l1: 9.90128
Early stopping, best iteration is:
[473]	train's l1: 8.24126	valid's l1: 9.8942
function: 'fit' finished in  11.30 seconds
function: 'predict' starting
function: 'predict' finished in  0.11 seconds
regressi

...

function: 'fit' starting
entered fit, x shape: (13308, 1792)
Training until validation scores don't improve for 100 rounds
[100]	train's binary_error: 0.249173	valid's binary_error: 0.338443
Early stopping, best iteration is:
[16]	train's binary_error: 0.273419	valid's binary_error: 0.329726
function: 'fit' finished in  4.90 seconds
function: 'predict' starting
function: 'predict' finished in  0.28 seconds
classification 0.6584761045987376
function: 'fit' starting
entered fit, x shape: (13308, 1792)
Training until validation scores don't improve for 100 rounds
[100]	train's l1: 9.69408	valid's l1: 10.2127
[200]	train's l1: 9.07283	valid's l1: 9.96594
[300]	train's l1: 8.64581	valid's l1: 9.90748
[400]	train's l1: 8.27729	valid's l1: 9.89393
[500]	train's l1: 7.94408	valid's l1: 9.88523
Early stopping, best iteration is:
[492]	train's l1: 7.9705	valid's l1: 9.88399
function: 'fit' finished in  18.88 seconds
function: 'predict' starting
function: 'predict' finished in  0.32 seconds
regre

/home/td/anaconda3/envs/sports_predictor/lib/python3.7/site-packages/scipy/stats/_stats_mstats_common.py:130: RuntimeWarning: invalid value encountered in double_scalars
  slope = r_num / ssxm
/home/td/anaconda3/envs/sports_predictor/lib/python3.7/site-packages/scipy/stats/_stats_mstats_common.py:142: RuntimeWarning: divide by zero encountered in double_scalars
  sterrest = np.sqrt((1 - r**2) * ssym / ssxm / df)


function: 'fit' starting
entered fit, x shape: (13308, 1138)
Training until validation scores don't improve for 100 rounds
[100]	train's binary_error: 0.250075	valid's binary_error: 0.337241
[200]	train's binary_error: 0.229937	valid's binary_error: 0.339645
Early stopping, best iteration is:
[126]	train's binary_error: 0.245466	valid's binary_error: 0.333333
function: 'fit' finished in  5.59 seconds
function: 'predict' starting
function: 'predict' finished in  0.17 seconds
classification 0.6647880973850315
function: 'fit' starting
entered fit, x shape: (13308, 1138)
Training until validation scores don't improve for 100 rounds
[100]	train's l1: 9.71464	valid's l1: 10.236
[200]	train's l1: 9.09405	valid's l1: 9.98784
[300]	train's l1: 8.66763	valid's l1: 9.91724
[400]	train's l1: 8.29881	valid's l1: 9.89677
[500]	train's l1: 7.95609	valid's l1: 9.89947
Early stopping, best iteration is:
[462]	train's l1: 8.08441	valid's l1: 9.89214
function: 'fit' finished in  12.47 seconds
function: '

/home/td/anaconda3/envs/sports_predictor/lib/python3.7/site-packages/scipy/stats/_stats_mstats_common.py:130: RuntimeWarning: invalid value encountered in double_scalars
  slope = r_num / ssxm
/home/td/anaconda3/envs/sports_predictor/lib/python3.7/site-packages/scipy/stats/_stats_mstats_common.py:142: RuntimeWarning: divide by zero encountered in double_scalars
  sterrest = np.sqrt((1 - r**2) * ssym / ssxm / df)


function: 'fit' starting
entered fit, x shape: (13308, 2368)
Training until validation scores don't improve for 100 rounds
[100]	train's binary_error: 0.241759	valid's binary_error: 0.333033
Early stopping, best iteration is:
[24]	train's binary_error: 0.261898	valid's binary_error: 0.323114
function: 'fit' finished in  9.15 seconds
function: 'predict' starting
function: 'predict' finished in  0.37 seconds
classification 0.6580252479711451
function: 'fit' starting
entered fit, x shape: (13308, 2368)
Training until validation scores don't improve for 100 rounds
[100]	train's l1: 9.65035	valid's l1: 10.2039
[200]	train's l1: 9.0034	valid's l1: 9.9586
[300]	train's l1: 8.55566	valid's l1: 9.8996
[400]	train's l1: 8.16574	valid's l1: 9.89395
[500]	train's l1: 7.80686	valid's l1: 9.88708
[600]	train's l1: 7.47491	valid's l1: 9.8879
Early stopping, best iteration is:
[566]	train's l1: 7.58542	valid's l1: 9.8836
function: 'fit' finished in  39.13 seconds
function: 'predict' starting
function:

In [16]:
# for p in possible_solutions:
    #     col_choice_dict = get_random_params_given_col_list(baseline_classification, p)

for i in range(1000000):
    print(f'results: {len(results)}')
    
    col_choice_dict = get_random_params_and_col_list(baseline_classification)
    if len(col_choice_dict['output_columns']) < 12 or len(col_choice_dict['output_columns']) > 4000:
        print(f'skipping params with {len(col_choice_dict["output_columns"])} columns')
        continue

    temp_classification, _ = evaluate_features(feature_df, col_choice_dict['output_columns'], 'win', 'classification')
    temp_regression, _ = evaluate_features(feature_df, col_choice_dict['output_columns'], 'score_diff', 'regression')
    
    col_choice_dict['win_loss_accuracy'] = temp_classification['model_performance'].min()
    col_choice_dict['points_estimation_r2'] = temp_regression['model_performance'].min()
    col_choice_dict['col_count'] = len(col_choice_dict['output_columns'])
    
    del col_choice_dict['output_columns']

    results.append(col_choice_dict)
    pd.DataFrame.from_dict(results).sort_values('win_loss_accuracy', ascending = False).to_csv('/media/td/Samsung_T5/sports/nba/feature_reduction2.csv', index = False, sep = '|')
#     print(results)

results: 2261
function: 'fit' starting
entered fit, x shape: (13308, 450)
Training until validation scores don't improve for 100 rounds
[100]	train's binary_error: 0.257189	valid's binary_error: 0.34716
[200]	train's binary_error: 0.241759	valid's binary_error: 0.345056
Early stopping, best iteration is:
[122]	train's binary_error: 0.253582	valid's binary_error: 0.341749
function: 'fit' finished in  4.24 seconds
function: 'predict' starting
function: 'predict' finished in  0.09 seconds
classification 0.6575743913435528
function: 'fit' starting
entered fit, x shape: (13308, 450)
Training until validation scores don't improve for 100 rounds
[100]	train's l1: 9.83218	valid's l1: 10.3748
[200]	train's l1: 9.24394	valid's l1: 10.1306
[300]	train's l1: 8.84197	valid's l1: 10.0737
[400]	train's l1: 8.4893	valid's l1: 10.0655
[500]	train's l1: 8.16247	valid's l1: 10.0621
Early stopping, best iteration is:
[461]	train's l1: 8.28587	valid's l1: 10.0607
function: 'fit' finished in  11.38 seconds


/home/td/anaconda3/envs/sports_predictor/lib/python3.7/site-packages/scipy/stats/_stats_mstats_common.py:130: RuntimeWarning: invalid value encountered in double_scalars
  slope = r_num / ssxm
/home/td/anaconda3/envs/sports_predictor/lib/python3.7/site-packages/scipy/stats/_stats_mstats_common.py:142: RuntimeWarning: divide by zero encountered in double_scalars
  sterrest = np.sqrt((1 - r**2) * ssym / ssxm / df)


function: 'fit' starting
entered fit, x shape: (13308, 226)
Training until validation scores don't improve for 100 rounds
[100]	train's binary_error: 0.275824	valid's binary_error: 0.348061
Early stopping, best iteration is:
[7]	train's binary_error: 0.303577	valid's binary_error: 0.343252
function: 'fit' finished in  1.94 seconds
function: 'predict' starting
function: 'predict' finished in  0.08 seconds
classification 0.6444995491433724
function: 'fit' starting
entered fit, x shape: (13308, 226)
Training until validation scores don't improve for 100 rounds
[100]	train's l1: 9.82284	valid's l1: 10.303
[200]	train's l1: 9.30034	valid's l1: 10.066
[300]	train's l1: 8.95833	valid's l1: 10.0022
[400]	train's l1: 8.67134	valid's l1: 9.98137
[500]	train's l1: 8.41113	valid's l1: 9.98242
Early stopping, best iteration is:
[427]	train's l1: 8.59934	valid's l1: 9.98024
function: 'fit' finished in  6.57 seconds
function: 'predict' starting
function: 'predict' finished in  0.06 seconds
regression

/home/td/anaconda3/envs/sports_predictor/lib/python3.7/site-packages/scipy/stats/_stats_mstats_common.py:130: RuntimeWarning: invalid value encountered in double_scalars
  slope = r_num / ssxm
/home/td/anaconda3/envs/sports_predictor/lib/python3.7/site-packages/scipy/stats/_stats_mstats_common.py:142: RuntimeWarning: divide by zero encountered in double_scalars
  sterrest = np.sqrt((1 - r**2) * ssym / ssxm / df)


function: 'fit' starting
entered fit, x shape: (13308, 510)
Training until validation scores don't improve for 100 rounds
[100]	train's binary_error: 0.255586	valid's binary_error: 0.344755
[200]	train's binary_error: 0.23665	valid's binary_error: 0.341749
Early stopping, best iteration is:
[112]	train's binary_error: 0.253181	valid's binary_error: 0.341148
function: 'fit' finished in  6.45 seconds
function: 'predict' starting
function: 'predict' finished in  0.10 seconds
classification 0.6442741208295762
function: 'fit' starting
entered fit, x shape: (13308, 510)
Training until validation scores don't improve for 100 rounds
[100]	train's l1: 9.82094	valid's l1: 10.3825
[200]	train's l1: 9.2263	valid's l1: 10.144
[300]	train's l1: 8.81151	valid's l1: 10.0805
[400]	train's l1: 8.4565	valid's l1: 10.0523
[500]	train's l1: 8.13186	valid's l1: 10.0428
[600]	train's l1: 7.83034	valid's l1: 10.0411
Early stopping, best iteration is:
[541]	train's l1: 8.00653	valid's l1: 10.0405
function: 'fi

...

/home/td/anaconda3/envs/sports_predictor/lib/python3.7/site-packages/scipy/stats/_stats_mstats_common.py:130: RuntimeWarning: invalid value encountered in double_scalars
  slope = r_num / ssxm
/home/td/anaconda3/envs/sports_predictor/lib/python3.7/site-packages/scipy/stats/_stats_mstats_common.py:142: RuntimeWarning: divide by zero encountered in double_scalars
  sterrest = np.sqrt((1 - r**2) * ssym / ssxm / df)


function: 'fit' starting
entered fit, x shape: (13308, 1190)
Training until validation scores don't improve for 100 rounds
[100]	train's binary_error: 0.255085	valid's binary_error: 0.345056
Early stopping, best iteration is:
[50]	train's binary_error: 0.265705	valid's binary_error: 0.339645
function: 'fit' finished in  9.35 seconds
function: 'predict' starting
function: 'predict' finished in  0.18 seconds
classification 0.6562218214607755
function: 'fit' starting
entered fit, x shape: (13308, 1190)
Training until validation scores don't improve for 100 rounds
[100]	train's l1: 9.75095	valid's l1: 10.31
[200]	train's l1: 9.1206	valid's l1: 10.0586
[300]	train's l1: 8.67371	valid's l1: 9.9959
[400]	train's l1: 8.28793	valid's l1: 9.97546
[500]	train's l1: 7.93717	valid's l1: 9.97288
Early stopping, best iteration is:
[489]	train's l1: 7.97489	valid's l1: 9.97198
function: 'fit' finished in  42.13 seconds
function: 'predict' starting
function: 'predict' finished in  0.23 seconds
regressi

/home/td/anaconda3/envs/sports_predictor/lib/python3.7/site-packages/scipy/stats/_stats_mstats_common.py:130: RuntimeWarning: invalid value encountered in double_scalars
  slope = r_num / ssxm
/home/td/anaconda3/envs/sports_predictor/lib/python3.7/site-packages/scipy/stats/_stats_mstats_common.py:142: RuntimeWarning: divide by zero encountered in double_scalars
  sterrest = np.sqrt((1 - r**2) * ssym / ssxm / df)


function: 'fit' starting
entered fit, x shape: (13308, 2170)
Training until validation scores don't improve for 100 rounds


KeyboardInterrupt: 

In [ ]:
pd.DataFrame.from_dict(results).sort_values('win_loss_accuracy', ascending = False)

In [ ]:
general_cols = baseline_classification[baseline_classification['feature_type'] == 'general']['column'].tolist()

In [ ]:
baseline_classification.groupby(['stat_type'])[['normalized_feature_importance']].sum().sort_values('normalized_feature_importance', ascending = False).head(100)

In [ ]:
baseline_classification.groupby(['aggregation_window_size'])[['normalized_feature_importance']].sum().sort_values('normalized_feature_importance', ascending = False).head(100)

In [ ]:
baseline_classification.groupby(['team_aggregation_function'])[['normalized_feature_importance']].sum().sort_values('normalized_feature_importance', ascending = False).head(100)

In [ ]:
baseline_classification.groupby(['player_aggregation_function'])[['normalized_feature_importance']].sum().sort_values('normalized_feature_importance', ascending = False).head(100)

In [ ]:
baseline_regression.groupby(['stat_type'])[['normalized_feature_importance']].sum().sort_values('normalized_feature_importance', ascending = False).head(100)

In [ ]:
baseline_regression.groupby(['aggregation_window_size'])[['normalized_feature_importance']].sum().sort_values('normalized_feature_importance', ascending = False).head(100)

In [ ]:
baseline_regression.groupby(['team_aggregation_function'])[['normalized_feature_importance']].sum().sort_values('normalized_feature_importance', ascending = False).head(100)

In [ ]:
baseline_regression.groupby(['player_aggregation_function'])[['normalized_feature_importance']].sum().sort_values('normalized_feature_importance', ascending = False).head(100)

In [ ]:
baseline_classification.groupby(['player_aggregation_function', 'stat_type'])[['normalized_feature_importance']].sum().sort_values('normalized_feature_importance', ascending = False).head(100)

In [ ]:
baseline_regression.groupby(['player_aggregation_function', 'stat_type'])[['normalized_feature_importance']].sum().sort_values('normalized_feature_importance', ascending = False).head(100)

In [ ]:
baseline_classification.groupby(['team_aggregation_function', 'stat_type'])[['normalized_feature_importance']].sum().sort_values('normalized_feature_importance', ascending = False).head(100)

In [ ]:
baseline_regression.groupby(['team_aggregation_function', 'stat_type'])[['normalized_feature_importance']].sum().sort_values('normalized_feature_importance', ascending = False).head(100)

In [ ]:
selected_window_sizes = []
selected_team_aggregation_functions = []
selected_player_aggregation_functions = []
selected_stat_types = []

cols = baseline_classification[(baseline_classification['aggregation_window_size'].isin(selected_window_sizes))&
                        (baseline_classification['team_aggregation_function'].isin(selected_team_aggregation_functions))&
                        (baseline_classification['player_aggregation_function'].isin(selected_player_aggregation_functions))&
                        (baseline_classification['stat_type'].isin(selected_stat_types))]['column'].tolist()

other_cols = []


In [ ]:
import re

feature_classification_df_dict_comb2 = dict()
feature_classification_dfs_comb2 = list()
classification_pred_df_comb2 = pd.DataFrame()

feature_regression_df_dict_comb2 = dict()
feature_regression_dfs_comb2 = list()
regression_pred_df_comb2 = pd.DataFrame()

expressions = []
for i in ['avg', 'var', 'skew', 'max', 'median']:
    for j in ['avg', 'var', 'skew', 'max', 'median']:
        expressions.append((i, j))

for c, n in enumerate(expressions):
    print(f'c: {c}, expressions:{len(expressions)},  n: {n}')
    print(len(feature_df.columns.tolist()))
    cols = [i for i in feature_df.columns if re.findall(f'{n[0]}_player_stats_aggregated_[A-Za-z0-9_]+{n[1]}', i) and i not in ['win', 'score_diff', 'key']]
    print(len(cols))
    if len(cols) < 10:
        continue
    temp_df_classification, preds_classification = evaluate_features(feature_df, cols, 'win', 'classification')
    temp_df_regression, preds_regression = evaluate_features(feature_df, cols, 'score_diff', 'regression')

    temp_df_classification['player_aggregation'] = n[1]
    temp_df_classification['team_aggregation'] = n[0]
    classification_pred_df_comb2[n] = preds_classification
    feature_classification_df_dict_comb2[n] = temp_df_classification
    feature_classification_dfs_comb2.append(temp_df_classification)
    
    temp_df_regression['player_aggregation'] = n[1]
    temp_df_regression['team_aggregation'] = n[0]
    regression_pred_df_comb2[n] = preds_regression
    feature_regression_df_dict_comb2[n] = temp_df_regression
    feature_regression_dfs_comb2.append(temp_df_regression)

feature_classification_dfs_comb2 = pd.concat(feature_classification_dfs_comb2)
feature_regression_dfs_comb2 = pd.concat(feature_regression_dfs_comb2)

In [ ]:
feature_classification_dfs_comb2.groupby(['player_aggregation', 'team_aggregation'])[['model_performance']].mean().sort_values('model_performance', ascending = False)

In [ ]:
feature_classification_dfs_comb2.groupby(['player_aggregation'])[['model_performance']].max().sort_values('model_performance', ascending = False)

In [ ]:
feature_classification_dfs_comb2.groupby(['player_aggregation'])[['model_performance']].min().sort_values('model_performance', ascending = False)

In [ ]:
feature_classification_dfs_comb2.groupby(['team_aggregation'])[['model_performance']].max().sort_values('model_performance', ascending = False)

In [ ]:
feature_classification_dfs_comb2.groupby(['team_aggregation'])[['model_performance']].min().sort_values('model_performance', ascending = False)

In [ ]:
import re

feature_classification_df_dict_comb = dict()
feature_classification_dfs_comb = list()
classification_pred_df_comb = pd.DataFrame()

feature_regression_df_dict_comb = dict()
feature_regression_dfs_comb = list()
regression_pred_df_comb = pd.DataFrame()

expressions = []
for i in ['avg', 'var', 'skew', 'max', 'median']:
    for j in ['avg', 'var', 'skew', 'max', 'median']:
        for k in [1, 3, 5, 7, 12, 20, 50, 100]:
            expressions.append((i, j, k))

for c, n in enumerate(expressions):
    print(f'c: {c}, expressions:{len(expressions)},  n: {n}')
    print(len(feature_df.columns.tolist()))
    cols = [i for i in feature_df.columns if 'team_aggregate_past_{}_'.format(n[2]) in i and re.findall(f'{n[0]}_player_stats_aggregated_[A-Za-z0-9_]+{n[1]}', i) and i not in ['win', 'score_diff', 'key']]
    print(len(cols))
    if len(cols) < 10:
        continue
    temp_df_classification, preds_classification = evaluate_features(feature_df, cols, 'win', 'classification')
    temp_df_regression, preds_regression = evaluate_features(feature_df, cols, 'score_diff', 'regression')

    temp_df_classification['player_aggregation'] = n[1]
    temp_df_classification['team_aggregation'] = n[0]
    temp_df_classification['aggregation_window_size'] = n[2]
    classification_pred_df_comb[n] = preds_classification
    feature_classification_df_dict_comb[n] = temp_df_classification
    feature_classification_dfs_comb.append(temp_df_classification)
    
    temp_df_regression['player_aggregation'] = n[1]
    temp_df_regression['team_aggregation'] = n[0]
    temp_df_classification['aggregation_window_size'] = n[2]

    regression_pred_df_comb[n] = preds_regression
    feature_regression_df_dict_comb[n] = temp_df_regression
    feature_regression_dfs_comb.append(temp_df_regression)

feature_classification_dfs_comb = pd.concat(feature_classification_dfs_comb)
feature_regression_dfs_comb = pd.concat(feature_regression_dfs_comb)

In [ ]:
feature_classification_dfs_comb.groupby(['player_aggregation', 'team_aggregation', 'aggregation_window_size'])[['model_performance']].mean().sort_values('model_performance', ascending = False)

In [ ]:
feature_classification_dfs_comb.groupby(['player_aggregation'])[['model_performance']].max().sort_values('model_performance', ascending = False)

In [ ]:
feature_classification_dfs_comb.groupby(['player_aggregation'])[['model_performance']].min().sort_values('model_performance', ascending = False)

In [ ]:
feature_classification_dfs_comb.groupby(['team_aggregation'])[['model_performance']].max().sort_values('model_performance', ascending = False)

In [ ]:
feature_classification_dfs_comb.groupby(['team_aggregation'])[['model_performance']].min().sort_values('model_performance', ascending = False)

In [ ]:
feature_classification_dfs_comb.groupby(['aggregation_window_size'])[['model_performance']].max().sort_values('model_performance', ascending = False)

In [ ]:
feature_classification_dfs_comb.groupby(['aggregation_window_size'])[['model_performance']].max().sort_values('model_performance', ascending = False)

In [ ]:
import re

feature_classification_df_dict_comb3 = dict()
feature_classification_dfs_comb3 = list()
classification_pred_df_comb3 = pd.DataFrame()

feature_regression_df_dict_comb3 = dict()
feature_regression_dfs_comb3 = list()
regression_pred_df_comb3 = pd.DataFrame()

expressions = []
for i in ['avg', 'var', 'skew', 'max', 'median']:
    for j in ['avg', 'var', 'skew', 'max', 'median']:
        for k in ['ast', 'ast_pct', 'blk', 'blk_pct', 'def_rtg', 'drb', 'drb_pct', 'efg_pct',
                                      'fg', 'fg3', 'fg3_pct', 'fg3a', 'fg3a_per_fga_pct', 'fg_pct', 'fga', 'ft',
                                      'ft_pct', 'fta', 'fta_per_fga_pct', 'off_rtg', 'orb', 'orb_pct', 'pf',
                                      'pts', 'stl', 'stl_pct', 'tov', 'tov_pct', 'trb', 'trb_pct',
                                      'ts_pct', 'usg_pct', 'home', 'days_since_last_match']:
        expressions.append((i, j, k))

for c, n in enumerate(expressions):
    print(f'c: {c}, expressions:{len(expressions)},  n: {n}')
    print(len(feature_df.columns.tolist()))
    cols = [i for i in feature_df.columns if '_{}_'.format(k) in i and re.findall(f'{n[0]}_player_stats_aggregated_[A-Za-z0-9_]+{n[1]}', i) and i not in ['win', 'score_diff', 'key']]
    print(len(cols))
    if len(cols) < 10:
        continue
    temp_df_classification, preds_classification = evaluate_features(feature_df, cols, 'win', 'classification')
    temp_df_regression, preds_regression = evaluate_features(feature_df, cols, 'score_diff', 'regression')

    temp_df_classification['player_aggregation'] = n[1]
    temp_df_classification['team_aggregation'] = n[0]
    temp_df_classification['stat'] = n[2]
    classification_pred_df_comb3[n] = preds_classification
    feature_classification_df_dict_comb3[n] = temp_df_classification
    feature_classification_dfs_comb3.append(temp_df_classification)
    
    temp_df_regression['player_aggregation'] = n[1]
    temp_df_regression['team_aggregation'] = n[0]
    temp_df_classification['stat'] = n[2]
    regression_pred_df_comb3[n] = preds_regression
    feature_regression_df_dict_comb3[n] = temp_df_regression
    feature_regression_dfs_comb3.append(temp_df_regression)

feature_classification_dfs_comb3 = pd.concat(feature_classification_dfs_comb3)
feature_regression_dfs_comb3 = pd.concat(feature_regression_dfs_comb3)

In [ ]:
feature_classification_dfs_comb3.groupby(['player_aggregation', 'team_aggregation', 'stat'])[['model_performance']].mean().sort_values('model_performance', ascending = False)

In [ ]:
feature_classification_dfs_comb3.groupby(['player_aggregation'])[['model_performance']].max().sort_values('model_performance', ascending = False)

In [ ]:
feature_classification_dfs_comb3.groupby(['player_aggregation'])[['model_performance']].min().sort_values('model_performance', ascending = False)

In [ ]:
feature_classification_dfs_comb3.groupby(['team_aggregation'])[['model_performance']].max().sort_values('model_performance', ascending = False)

In [ ]:
feature_classification_dfs_comb3.groupby(['team_aggregation'])[['model_performance']].min().sort_values('model_performance', ascending = False)

In [ ]:
feature_classification_dfs_comb3.groupby(['stat'])[['model_performance']].max().sort_values('model_performance', ascending = False)

In [ ]:
feature_classification_dfs_comb3.groupby(['stat'])[['model_performance']].min().sort_values('model_performance', ascending = False)

In [ ]:
import re

feature_classification_df_dict_comb = dict()
feature_classification_dfs_comb = list()
classification_pred_df_comb = pd.DataFrame()

feature_regression_df_dict_comb = dict()
feature_regression_dfs_comb = list()
regression_pred_df_comb = pd.DataFrame()

expressions = []
for i in ['avg', 'var', 'skew', 'max', 'median']:
    for j in ['avg', 'var', 'skew', 'max', 'median']:
        expressions.append((i, j))

for n in expressions:
    print(f'n: {n}')
    print(len(feature_df.columns.tolist()))
    cols = [i for i in feature_df.columns if not re.findall(f'{n[0]}_player_stats_aggregated_[A-Za-z0-9_]+{n[1]}', i) and i not in ['win', 'score_diff', 'key']]
    print(len(cols))
    temp_df_classification, preds_classification = evaluate_features(feature_df, cols, 'win', 'classification')
    temp_df_regression, preds_regression = evaluate_features(feature_df, cols, 'score_diff', 'regression')

    temp_df_classification['skipped_player_aggregation'] = n[1]
    temp_df_classification['skipped_team_aggregation'] = n[0]
    classification_pred_df_comb[n] = preds_classification
    feature_classification_df_dict_comb[n] = temp_df_classification
    feature_classification_dfs_comb.append(temp_df_classification)
    
    temp_df_regression['skipped_player_aggregation'] = n[1]
    temp_df_regression['skipped_team_aggregation'] = n[0]
    regression_pred_df_comb[n] = preds_regression
    feature_regression_df_dict_comb[n] = temp_df_regression
    feature_regression_dfs_comb.append(temp_df_regression)

feature_classification_dfs_comb = pd.concat(feature_classification_dfs_comb)
feature_regression_dfs_comb = pd.concat(feature_regression_dfs_comb)

In [ ]:
feature_classification_dfs_comb.groupby(['skipped_player_aggregation', 'skipped_team_aggregation'])[['model_performance']].mean()

In [ ]:
feature_classification_dfs_comb.groupby(['skipped_player_aggregation'])[['model_performance']].max()

In [ ]:
feature_classification_dfs_comb.groupby(['skipped_player_aggregation'])[['model_performance']].min()

In [ ]:
feature_classification_dfs_comb.groupby(['skipped_team_aggregation'])[['model_performance']].max()

In [ ]:
feature_classification_dfs_comb.groupby(['skipped_team_aggregation'])[['model_performance']].min()

In [ ]:
feature_regression_dfs_comb.groupby(['skipped_player_aggregation', 'skipped_team_aggregation'])[['model_performance']].mean()

In [ ]:
feature_regression_dfs_comb.groupby(['skipped_player_aggregation'])[['model_performance']].max()

In [ ]:
feature_regression_dfs_comb.groupby(['skipped_player_aggregation'])[['model_performance']].min()

In [ ]:
feature_regression_dfs_comb.groupby(['skipped_team_aggregation'])[['model_performance']].max()

In [ ]:
feature_regression_dfs_comb.groupby(['skipped_team_aggregation'])[['model_performance']].min()

In [ ]:
import re

feature_classification_df_dict_team = dict()
feature_classification_dfs_team = list()
classification_pred_df_team = pd.DataFrame()

feature_regression_df_dict_team = dict()
feature_regression_dfs_team = list()
regression_pred_df_team = pd.DataFrame()


for n in ['avg', 'var', 'skew', 'max', 'median']:
    print(f'n: {n}')
    print(len(feature_df.columns.tolist()))
    cols = [i for i in feature_df.columns if not re.findall(f'{n}_player_stats_aggregated_[A-Za-z0-9_]+', i) and i not in ['win', 'score_diff', 'key']]
    print(len(cols))
    temp_df_classification, preds_classification = evaluate_features(feature_df, cols, 'win', 'classification')
    temp_df_regression, preds_regression = evaluate_features(feature_df, cols, 'score_diff', 'regression')

    temp_df_classification['skipped_team_aggregation'] = n
    classification_pred_df_team[n] = preds_classification
    feature_classification_df_dict_team[n] = temp_df_classification
    feature_classification_dfs_team.append(temp_df_classification)
    
    temp_df_regression['skipped_team_aggregation'] = n
    regression_pred_df_team[n] = preds_regression
    feature_regression_df_dict_team[n] = temp_df_regression
    feature_regression_dfs_team.append(temp_df_regression)

feature_classification_dfs_team = pd.concat(feature_classification_dfs_team)
feature_regression_dfs_team = pd.concat(feature_regression_dfs_team)

In [ ]:
feature_classification_dfs_team.groupby(['skipped_team_aggregation'])[['model_performance']].max()

In [ ]:
feature_classification_dfs_team.groupby(['skipped_team_aggregation'])[['model_performance']].min()

In [ ]:
feature_regression_dfs_team.groupby(['skipped_team_aggregation'])[['model_performance']].max()

In [ ]:
feature_regression_dfs_team.groupby(['skipped_team_aggregation'])[['model_performance']].min()

In [ ]:
import re

feature_classification_df_dict_players = dict()
feature_classification_dfs_players = list()
classification_pred_df_players = pd.DataFrame()

feature_regression_df_dict_players = dict()
feature_regression_dfs_players = list()
regression_pred_df_players = pd.DataFrame()



for n in ['avg', 'var', 'skew', 'max', 'median']:
    print(f'n: {n}')
    print(len(feature_df.columns.tolist()))
    cols = [i for i in feature_df.columns if not re.findall(f'_player_stats_aggregated_[A-Za-z0-9_]+{n}', i) and i not in ['win', 'score_diff', 'key']]
    print(len(cols))
    temp_df_classification, preds_classification = evaluate_features(feature_df, cols, 'win', 'classification')
    temp_df_regression, preds_regression = evaluate_features(feature_df, cols, 'score_diff', 'regression')

    temp_df_classification['skipped_player_aggregation'] = n
    classification_pred_df_players[n] = preds_classification
    feature_classification_df_dict_players[n] = temp_df_classification
    feature_classification_dfs_players.append(temp_df_classification)
    
    temp_df_regression['skipped_player_aggregation'] = n
    regression_pred_df_players[n] = preds_regression
    feature_regression_df_dict_players[n] = temp_df_regression
    feature_regression_dfs_players.append(temp_df_regression)

feature_classification_dfs_players = pd.concat(feature_classification_dfs_players)
feature_regression_dfs_players = pd.concat(feature_regression_dfs_players)

In [ ]:
feature_classification_dfs_players.groupby(['skipped_player_aggregation'])[['model_performance']].max()

In [ ]:
feature_classification_dfs_players.groupby(['skipped_player_aggregation'])[['model_performance']].min()

In [ ]:
feature_regression_dfs_players.groupby(['skipped_player_aggregation'])[['model_performance']].max()

In [ ]:
feature_regression_dfs_players.groupby(['skipped_player_aggregation'])[['model_performance']].min()

In [ ]:
import re

feature_classification_df_dict_stats = dict()
feature_classification_dfs_stats = list()
classification_pred_df_stats = pd.DataFrame()

feature_regression_df_dict_players = dict()
feature_regression_dfs_players = list()
regression_pred_df_players = pd.DataFrame()



for n in ['ast', 'ast_pct', 'blk', 'blk_pct', 'def_rtg', 'drb', 'drb_pct', 'efg_pct',
                                      'fg', 'fg3', 'fg3_pct', 'fg3a', 'fg3a_per_fga_pct', 'fg_pct', 'fga', 'ft',
                                      'ft_pct', 'fta', 'fta_per_fga_pct', 'off_rtg', 'orb', 'orb_pct', 'pf',
                                      'pts', 'stl', 'stl_pct', 'tov', 'tov_pct', 'trb', 'trb_pct',
                                      'ts_pct', 'usg_pct', 'home', 'days_since_last_match']:
    print(f'n: {n}')
    print(len(feature_df.columns.tolist()))
    cols = [i for i in feature_df.columns if not re.findall(f'_player_stats_aggregated_[A-Za-z0-9_]+{n}', i) and i not in ['win', 'score_diff', 'key']]
    print(len(cols))
    temp_df_classification, preds_classification = evaluate_features(feature_df, cols, 'win', 'classification')
    temp_df_regression, preds_regression = evaluate_features(feature_df, cols, 'score_diff', 'regression')

    temp_df_classification['skipped_stat'] = n
    classification_pred_df_stats[n] = preds_classification
    feature_classification_df_dict_stats[n] = temp_df_classification
    feature_classification_dfs_stats.append(temp_df_classification)
    
    temp_df_regression['skipped_stat'] = n
    regression_pred_df_stats[n] = preds_regression
    feature_regression_df_dict_stats[n] = temp_df_regression
    feature_regression_dfs_stats.append(temp_df_regression)

feature_classification_dfs_stats = pd.concat(feature_classification_dfs_stats)
feature_regression_dfs_stats = pd.concat(feature_regression_dfs_stats)

In [ ]:
feature_classification_dfs_stats.groupby(['skipped_stat'])[['model_performance']].max()

In [ ]:
feature_classification_dfs_stats.groupby(['skipped_stat'])[['model_performance']].min()

In [ ]:
feature_regression_dfs_stats.groupby(['skipped_stat'])[['model_performance']].max()

In [ ]:
feature_regression_dfs_stats.groupby(['skipped_stat'])[['model_performance']].min()